# AADT tracker
### Combining OpenStreetMap with Mastra
*Written by - Rasmus Bergman rbvp20@student.aau.dk*

This is a notebook for combining the data from OpenStreetMap and Mastra to create an AADT attribute for each node in the OpenStreetMap graph

### **Prerequisits**
- Running mastra.ipynb
- Running OSM_fetching.ipynb

In [1]:
# Import libraries
import geopandas as gpd
import osmnx as ox
import constants as c
# Config
ox.settings.console_log = True
ox.settings.use_cache = True

# Parameters
ACCIDENT_RADIUS_METERS = 50

In [2]:
# Load full road network. This takes a long time.
road_network = ox.io.load_graphml(c.FULL_ROAD_NETWORK_PATH)

In [3]:
simplified_road_network = ox.simplify_graph(road_network, track_merged=True)

In [ ]:
# Load the mastra data and filter out all traffic counts which are not for motor vehicles
traffic_flow = gpd.read_file(c.MASTRA_PATH).to_crs("EPSG:25832")
traffic_flow = traffic_flow[traffic_flow["KOERETOEJSART"] == "MOTORKTJ" and traffic_flow["AAR"] >= 2018]\
    [["geometry", "AADT", "AAR"]]\
    .sort_values("AAR", ascending=False)
        

In [8]:
edges = ox.graph_to_gdfs(simplified_road_network, nodes=False)

In [ ]:
# Join the traffic flow data to the road network
edges = edges.to_crs("EPSG:25832")
gpd.sjoin_nearest(traffic_flow, edges)\
    .explode("merged_edges")\
    .explode("merged_edges")\
    .drop_duplicates(subset="merged_edges", keep="first") \
    [["merged_edges", "AADT"]]\
    .rename(columns={"merged_edges": "osmid"})\
    .to_csv(c.NODE_CSV_PATH, index=False)
